In [1]:
!pip install -qqq datasets==3.2.0 transformers==4.47.1 trl==0.14.0 peft==0.14.0 accelerate==1.2.1 bitsandbytes==0.45.2 wandb==0.19.7 --progress-bar off
!pip install -qqq flash-attn --no-build-isolation --progress-bar off

In [1]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer


/home/arthur/miniconda3/envs/agent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 03-09 22:35:20 __init__.py:183] Automatically detected platform cuda.


2025-03-09 22:35:21,069	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


# Wandb Initialization

In [2]:
import wandb

# Login
wandb.login()





wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: arthur-edmond-pro (arthur-edmond-perso) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Dataset

In [3]:
dataset = load_dataset("mlabonne/smoltldr")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 200
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 200
    })
})


In [4]:
model_id = "HuggingFaceTB/SmolLM-135M-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation="flash_attention_2",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# LORA

In [5]:
# Load LoRA
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
)
model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

trainable params: 4,884,480 || all params: 139,399,488 || trainable%: 3.5039
None


# Reward function

In [6]:
# Reward function
ideal_length = 50


def reward_len(completions, **kwargs):
    return [-abs(ideal_length - len(completion)) for completion in completions]

In [7]:
# Training arguments
training_args = GRPOConfig(
    output_dir="GRPO",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    max_prompt_length=256,
    max_completion_length=48,
    num_generations=8,
    optim="adamw_8bit",
    num_train_epochs=1,
    bf16=True,
    report_to=["wandb"],
    remove_unused_columns=False,
    logging_steps=1,
)

In [8]:
# Trainer
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[reward_len],
    args=training_args,
    train_dataset=dataset["train"],
)
# Train model
wandb.init(project="GRPO")
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
  0%|          | 1/2000 [00:02<1:29:00,  2.67s/it]

{'loss': -0.0, 'grad_norm': 0.2670714557170868, 'learning_rate': 1.9990000000000003e-05, 'completion_length': 48.0, 'rewards/reward_len': -155.5, 'reward': -155.5, 'reward_std': 22.97825050354004, 'kl': 0.0, 'epoch': 0.0}


  0%|          | 2/2000 [00:04<1:05:09,  1.96s/it]

{'loss': 0.0, 'grad_norm': 0.31133097410202026, 'learning_rate': 1.9980000000000002e-05, 'completion_length': 46.75, 'rewards/reward_len': -150.5, 'reward': -150.5, 'reward_std': 25.433385848999023, 'kl': 0.0009569795802235603, 'epoch': 0.0}


  0%|          | 3/2000 [00:05<57:47,  1.74s/it]  

...

  3%|▎         | 65/2000 [01:38<47:15,  1.47s/it]

{'loss': 0.0001, 'grad_norm': 0.47261181473731995, 'learning_rate': 1.9350000000000003e-05, 'completion_length': 40.5, 'rewards/reward_len': -79.625, 'reward': -79.625, 'reward_std': 49.64714813232422, 'kl': 0.002635023556649685, 'epoch': 0.03}


  3%|▎         | 66/2000 [01:40<47:26,  1.47s/it]

{'loss': 0.0001, 'grad_norm': 0.367867112159729, 'learning_rate': 1.934e-05, 'completion_length': 45.25, 'rewards/reward_len': -121.75, 'reward': -121.75, 'reward_std': 43.676570892333984, 'kl': 0.0034080822952091694, 'epoch': 0.03}


KeyboardInterrupt: 

## Save the model

In [10]:
merged_model = trainer.model.merge_and_unload()
merged_model.push_to_hub(
    "SmolGRPO-135M", private=False, tags=["GRPO", "Reasoning-Course"]
)

model.safetensors: 100%|██████████| 269M/269M [01:13<00:00, 3.64MB/s]


CommitInfo(commit_url='https://huggingface.co/Shumatsurontek/SmolGRPO-135M/commit/a8eb7466fd6fe16ea9e20346c375a9f42473be2d', commit_message='Upload LlamaForCausalLM', commit_description='', oid='a8eb7466fd6fe16ea9e20346c375a9f42473be2d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Shumatsurontek/SmolGRPO-135M', endpoint='https://huggingface.co', repo_type='model', repo_id='Shumatsurontek/SmolGRPO-135M'), pr_revision=None, pr_num=None)